In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd

REPO_ROOT = Path.cwd().parent.parent
BACKEND_DIR = REPO_ROOT / "backend"
sys.path.insert(0, str(BACKEND_DIR))

from data_engine.yfinance_fetcher import YFinanceFetcher
from data_engine.forecasting.prophet_model import ProphetForecaster

In [2]:
fetcher = YFinanceFetcher()
SYMBOL = "BTC-USD"
INTERVAL = "1wk"
PERIOD = "5y"

df = fetcher.fetch_history(SYMBOL, interval=INTERVAL, period=PERIOD)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp")

prices = df.set_index("timestamp")["close"].astype(float).dropna()
len(prices), prices.head()

(261,
 timestamp
 2021-02-22 00:00:00+00:00    45137.769531
 2021-03-01 00:00:00+00:00    51206.691406
 2021-03-08 00:00:00+00:00    59302.316406
 2021-03-15 00:00:00+00:00    57523.421875
 2021-03-22 00:00:00+00:00    55950.746094
 Name: close, dtype: float64)

In [3]:
TEST_SIZE = 30

def backtest_one_step(prices_full: pd.Series, test_size: int, model_factory, min_train: int):
    preds = []
    split_idx = len(prices_full) - test_size

    for i in range(split_idx, len(prices_full)):
        train = prices_full.iloc[:i]
        actual = float(prices_full.iloc[i])
        ts = prices_full.index[i]

        if len(train) < min_train:
            continue

        model = model_factory()
        model.fit(train)
        fc = model.forecast(periods=1)
        yhat = float(fc["point_forecast"][0])

        preds.append({"timestamp": ts, "y_true": actual, "y_pred": yhat})

    return pd.DataFrame(preds)

pred_prophet = backtest_one_step(
    prices, TEST_SIZE,
    model_factory=lambda: ProphetForecaster(confidence_level=0.95),
    min_train=10
)

pred_prophet.head()

c:\Users\khanh\OneDrive - GUSCanada\Documents\capstone_project_unfc\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
10:57:20 - cmdstanpy - INFO - Chain [1] start processing
10:57:25 - cmdstanpy - INFO - Chain [1] done processing
10:57:25 - cmdstanpy - INFO - Chain [1] start processing
10:57:25 - cmdstanpy - INFO - Chain [1] done processing
10:57:26 - cmdstanpy - INFO - Chain [1] start processing
10:57:26 - cmdstanpy - INFO - Chain [1] done processing
10:57:26 - cmdstanpy - INFO - Chain [1] start processing
10:57:26 - cmdstanpy - INFO - Chain [1] done processing
10:57:27 - cmdstanpy - INFO - Chain [1] start processing
10:57:27 - cmdstanpy - INFO - Chain [1] done processing
10:57:27 - cmdstanpy - INFO - Chain [1] start processing
10:57:27 - cmdstanpy - IN

,timestamp,y_true,y_pred
0,2025-07-28 00:00:00+00:00,114217.671875,105570.9364
1,2025-08-04 00:00:00+00:00,119306.757812,106386.6091
2,2025-08-11 00:00:00+00:00,117453.062500,107804.1433
3,2025-08-18 00:00:00+00:00,113458.429688,108712.6282
4,2025-08-25 00:00:00+00:00,108236.710938,108786.4148


In [4]:
OUTDIR = REPO_ROOT / "model" / "notebooks" / "artifacts"
OUTDIR.mkdir(parents=True, exist_ok=True)

path = OUTDIR / "pred_prophet_weekly_price.parquet"
pred_prophet.to_parquet(path, index=False)

print("Saved:", path)
print("Rows:", len(pred_prophet))

Saved: c:\Users\khanh\OneDrive - GUSCanada\Documents\capstone_project_unfc\model\notebooks\artifacts\pred_prophet_weekly_price.parquet
Rows: 30
